# OBTENÇÃO DOS MELHORES CLIENTES PARA UM CERTO NUMERO DE LOJAS

### Passo 1 - Encontrar o codigo das lojas no SCBI.

In [ ]:
# A PARTIR DESSA QUERY NO SCBI PUDEMOS COMPARAR OS NOMES DAS FILIAIS ELETROSSOM COM SEUS RESPECTIVOS CODIGOS.

SELECT 
    LOJA_LOJ_NOM_FANTASIA, *
FROM 
    [scbi].[st1].[st1_a_loja_adc_loj_fincdcdbs] as loj
WHERE
    LOJA_RED_LOJ_COD = 2975

### Passo 2 - Utilizando os codigos das lojas, gerar uma lista com todos os CPFs que originaram contratos nessas lojas em um determinado periodo.

In [ ]:
# QUERY QUE APONTA PARA OS CONTRATOS QUE FORAM ORIGINADOS EM UMA DADA LOJA
# Nome do arquivo gerado: cpfs_clientes_das_lojas_X.csv

DECLARE @parDeSafra datetime = '2017-01-01',
        @parAteSafra datetime = '2019-12-31',
        @parDatReferencia datetime = '2019-12-31'

SELECT
    LOJA_LOJ_COD,
    (con.cont_bas_cgccpf + con.cont_dig_cgccpf) AS CPF
FROM
    [scbi].[st1].[st1_a_cont_adc_con_fincdcdbs] con
        JOIN [scbi].[st1].[st1_a_loja_adc_loj_fincdcdbs] loj
            ON (con.cont_loj_cod = loj.loja_loj_cod AND con.cont_emp_cod = loj.loja_emp_cod)
        JOIN [scbi].[st1].[st1_a_rede_adc_red_loj_fincdcdbs] red
            ON (red.rede_red_loj_cod = loj.loja_red_loj_cod AND red.rede_emp_cod = loj.loja_emp_cod)
WHERE
    con.cont_con_sta NOT IN ('C')
    AND con.cont_con_dta_ini BETWEEN @parDeSafra AND @parAteSafra
    AND LOJA_LOJ_COD IN (29750056, 29750026, 29750044, 29750085, 29750047,
                         29750064, 29750029, 29750002, 29750021, 29750001)

### Passo 3 - Gerar o número de contratos de cada CPF em um determinado periodo.

In [ ]:
# QUERY DE CPFs POR CONTRATOS LIQUIDADOS
# Nome do arquivo gerado: cpfs_contratos_liq.csv

DECLARE @parDeSafra datetime = '2017-01-01',
        @parAteSafra datetime = '2019-12-31',
        @liqReferenciaInicial datetime = '2017-01-01',
        @liqReferenciaFinal datetime = '2019-12-31'

SELECT
    CONT_BAS_CGCCPF as cpf,
    COUNT(CONT_BAS_CGCCPF) as numr_contratos_liquidados
FROM
    [scbi].[st1].[st1_a_cont_adc_con_fincdcdbs]
WHERE
    CONT_CON_DTA_LIQ BETWEEN @liqReferenciaInicial AND @liqReferenciaFinal
    AND CONT_CON_DTA_INI BETWEEN @parDeSafra AND @parAteSafra
GROUP BY
    CONT_BAS_CGCCPF


### Passo 4: Executar o código python.

In [34]:
import pandas as pd
import dwsemear

import datetime
import collections
import postgresemear as sql

# dwsemear.atualize_metadados()

In [35]:
def retira_dv_cpf(cpf):
    cpf_str = str(cpf)
    return int(cpf_str[0:-2])

def mapeamento_loja(codigo):
    if codigo == 29750056:
        return 'Filial 62 Paracatu'
    elif codigo == 29750026:
        return 'Filial 28  Nova Serrana'
    elif codigo == 29750044:
        return 'Filial 46  Montes Claros'
    elif codigo == 29750085:
        return 'Filial 116 Vitória da Conquista'
    elif codigo == 29750047:
        return 'Filial 49  Uberlândia'
    elif codigo == 29750064:
        return 'Filial 71  Patos de Minas'
    elif codigo == 29750029:
        return 'Filial 31  Jataí'
    elif codigo == 29750002:
        return 'Filial 02  Uberaba'
    elif codigo == 29750021:
        return 'Filial 23  Sete Lagoas'
    elif codigo == 29750001:
        return 'Filial 01  Goiania'
    

In [36]:
file = 'cpfs_clientes_das_lojas_X.csv'
cpf_loja = pd.read_csv(file, sep=';')
file = 'cpfs_contratos_liq.csv'
cpfs_liq = pd.read_csv(file, sep=';')

parametros = (datetime.datetime(2019,9,30),)
dados_pdd = sql.query_para_pandas(
                          '''
                          SELECT
                              *
                          FROM 
                              pdd
                          WHERE 
                              pdd.data_referencia = %s
                          ''',
                          parametros)
dados_pdd.columns = dwsemear.pegue_colunas('pdd')

# CPFs de cada loja sem o DV

In [37]:
cpf_loja['cpf_sem_dv'] = cpf_loja.CPF.apply(retira_dv_cpf)
loja_sem_dv = [int(x) for x in cpf_loja.cpf_sem_dv.tolist()]
loja_sem_dv = set(cpf_loja.cpf_sem_dv.tolist())

# CPFs que tiveram liquidacoes sem o DV

In [38]:
liq_sem_dv = [int(x) for x in cpfs_liq.cpf.tolist()]
liq_sem_dv = set(liq_sem_dv)

# CPFs que estao atualmente em atraso sem o DV

In [39]:
cpfs_rating_A = []
cpfs_rating_abaixo_de_A = []
dados_pdd_antes_junho = dados_pdd[dados_pdd.data_originacao < datetime.datetime(2019,5,1)].copy()
for cpf in cpf_loja.CPF.tolist():
    corte_cpf = dados_pdd_antes_junho[dados_pdd_antes_junho.cpf == cpf]
    if len(corte_cpf) == 0:
        continue
    elif corte_cpf.rating_contabil_fim.iloc[0] == 'A':
        cpfs_rating_A.append(cpf)
    elif corte_cpf.rating_contabil_fim.iloc[0] != 'A':
        cpfs_rating_abaixo_de_A.append(cpf)        

In [40]:
cpfs_rating_baixo = set(list(map(retira_dv_cpf, cpfs_rating_abaixo_de_A)))

# COMBINACAO DOS CPFs

In [41]:
loja_e_liq = loja_sem_dv & liq_sem_dv

In [42]:
loja_liq_rating_A = loja_e_liq - cpfs_rating_baixo

# Concatenacao dos CPFs com a tabela das lojas

In [43]:
bons_clientes = []
for i in cpf_loja.index:
    if cpf_loja.cpf_sem_dv[i] in loja_liq_rating_A:
        bons_clientes.append(1)
    else:
        bons_clientes.append(0)

In [44]:
cpf_loja['bons'] = bons_clientes

In [45]:
cpf_loja['nome_filial'] = cpf_loja.LOJA_LOJ_COD.apply(mapeamento_loja)

In [46]:
cpf_loja = cpf_loja[cpf_loja.bons == 1].copy()

In [51]:
cpf_loja = cpf_loja.drop(['cpf_sem_dv','bons'],axis=1)

In [67]:
codigos_loja = list(set(cpf_loja.LOJA_LOJ_COD))

In [76]:
cpfs_bons_por_loja = {}
for codigo in codigos_loja:
    cpfs_codigo = list(set(cpf_loja[cpf_loja.LOJA_LOJ_COD == codigo].CPF))
    cpfs_bons_por_loja[codigo] = cpfs_codigo
    

In [86]:
tabela_cpfs_bons = []
for codigo in cpfs_bons_por_loja:
    for cpf in set(cpfs_bons_por_loja[codigo]):
        tabela_cpfs_bons.append(
            (codigo, 
              mapeamento_loja(codigo),
              cpf))

In [90]:
cpfs_lojas_selecionadas = pd.DataFrame(tabela_cpfs_bons)
cpfs_lojas_selecionadas.columns =['codigo_loja', 'nome_filial', 'cpf']
cpfs_lojas_selecionadas.to_csv('cpfs_lojas_selecionadas.csv', 
                               sep=';', 
                               encoding='latin-1', 
                               index=False)